In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]

In [2]:
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=3)
search.invoke("tell me about 2025 LA wildfire updates")

[{'url': 'https://en.wikipedia.org/wiki/January_2025_Southern_California_wildfires',
  'content': 'From January\xa07 to 31, 2025, a series of 14 destructive wildfires affected the Los Angeles metropolitan area and San Diego County in California, United States.[5] The fires were exacerbated by drought conditions, low humidity, a buildup of vegetation from the previous winter, and hurricane-force Santa Ana winds, which in some places reached 100 miles per hour (160\xa0km/h; 45\xa0m/s). On January\xa06, California governor Gavin Newsom stated he would assign 65 fire engines, 7 helicopters, 7 water tenders, and 109 workers to fighting wildfires that arose.[35] Mayor of Los Angeles Karen Bass warned residents that the windstorm could become one of the harshest in more than a decade, and cautioned them to avoid wind-downed power lines.[73] As Bass was out of the country for the inauguration of Ghanaian president John Mahama,[74] Council President Marqueece Harris-Dawson filled in as acting m

In [5]:
tools = [search]

In [6]:
tools

[TavilySearchResults(max_results=3)]

In [7]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [8]:
from langchain_core.messages import HumanMessage
response = agent_executor.invoke({"messages": [HumanMessage(content="tell me about 2025 LA wildfire updates")]})
response

{'messages': [HumanMessage(content='tell me about 2025 LA wildfire updates', id='1c7d0a90-fa09-4433-8419-f1e338054632'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_O9Xzk2YUChFeevZyxlJxPJJT', 'function': {'arguments': '{"query":"2025 Los Angeles wildfire updates"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-356588d4-7f9c-41f1-b4fd-5ed29d4dc6ef-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 Los Angeles wildfire updates'}, 'id': 'call_O9Xzk2YUChFeevZyxlJxPJJT', 'type': 'tool_call'}], usag

In [9]:
response["messages"]

[HumanMessage(content='tell me about 2025 LA wildfire updates', id='1c7d0a90-fa09-4433-8419-f1e338054632'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_O9Xzk2YUChFeevZyxlJxPJJT', 'function': {'arguments': '{"query":"2025 Los Angeles wildfire updates"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-356588d4-7f9c-41f1-b4fd-5ed29d4dc6ef-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 Los Angeles wildfire updates'}, 'id': 'call_O9Xzk2YUChFeevZyxlJxPJJT', 'type': 'tool_call'}], usage_metadata={'i

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [10]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [11]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "001"}}

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cKualkM8nAR2QMz6OK78gTeg', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 91, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-537a0571-3399-4077-8a12-452bb1a013b2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_cKualkM8nAR2QMz6OK78gTeg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 24, 'total_tokens': 115})]}}
----
{'tools': {'messages': [ToolMe

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='The top stars of the Spain team that won the 2024 soccer Eurocup include Mikel Oyarzabal, Nico Williams, and Lamine Yamal.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 932, 'total_tokens': 967, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7a7d13ff-fdec-4817-bb02-609232530fbf-0', usage_metadata={'input_tokens': 932, 'output_tokens': 35, 'total_tokens': 967})]}}
----


In [16]:
config = {"configurable": {"thread_id": "001"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='We were talking about the Spain national soccer team, which won the 2024 soccer Eurocup by defeating England in the final.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 981, 'total_tokens': 1009, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e72aa9d0-b83d-4782-8b77-fd2f4f8eb543-0', usage_metadata={'input_tokens': 981, 'output_tokens': 28, 'total_tokens': 1009})]}}
----
